In [80]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.layers as L
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



import numpy as np 
import pandas as pd

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

import re

In [81]:
df = pd.read_csv("../csv/data_preprocess.csv")

In [82]:
df.head()

,OriginalTweet,Sentiment
0,menyrbie phil gaan christ,Neutral
1,advice talk neighbour family exchange phone nu...,Positive
2,coronavirus australia woolworth give elderly d...,Positive
3,food stock one empty please panic enough food ...,Positive
4,me ready go supermarket couid outbreak i i par...,Negative


In [83]:
STOPWORDS = set(stopwords.words('english'))
STOPWORDS.add('u') #Konponketa 'u' hitza kontuan ez izateko.
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in set(STOPWORDS)])

df["OriginalTweet"] = df["OriginalTweet"].apply(lambda text:remove_stopwords(text))
df['OriginalTweet'] = df['OriginalTweet'].replace({'couid': 'covid'}, regex=True) #Hitza txarto eraldatua
df['OriginalTweet'] = df['OriginalTweet'].replace({'amp': ' '}, regex=True) #& arazoak

df['Sentiment'] = df['Sentiment'].replace(['Negative'],0)
df['Sentiment'] = df['Sentiment'].replace(['Neutral'],1)
df['Sentiment'] = df['Sentiment'].replace(['Positive'],2)

In [84]:
df

,OriginalTweet,Sentiment
0,menyrbie phil gaan christ,1
1,advice talk neighbour family exchange phone nu...,2
2,coronavirus australia woolworth give elderly d...,2
3,food stock one empty please panic enough food ...,2
4,ready go supermarket covid outbreak paranoid f...,0
...,...,...
41152,airline pilot offering stock supermarket shelf...,1
41153,response complaint provided citing covid relat...,0
41154,know getting tough kameronwilds rationing toil...,2
41155,wrong smell hand sanitizer starting turn coron...,1


In [85]:
tokenizer.fit_on_texts(df["OriginalTweet"])

X = tokenizer.texts_to_sequences(df["OriginalTweet"])

vocab_size = len(tokenizer.word_index)+1


In [86]:
vocab_size

41462

In [87]:
X = pad_sequences(X, padding='post')
X.shape

(41157, 38)

(37041, 38)

In [70]:

# hyper parameters
EPOCHS = 2
BATCH_SIZE = 32
embedding_dim = 16
units = 256

model = tf.keras.Sequential([
    L.Embedding(vocab_size, embedding_dim, input_length=X_train.shape[1]),
    L.Bidirectional(L.LSTM(units,return_sequences=True)),
    L.GlobalMaxPool1D(),
    L.Dropout(0.4),
    L.Dense(64, activation="relu"),
    L.Dropout(0.4),
    L.Dense(3)
])


model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',metrics=['accuracy']
             )

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 38, 16)            663248    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 38, 512)           559104    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                

In [71]:
history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.12, batch_size=BATCH_SIZE)

Epoch 1/2


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [32,3] and labels shape [96]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-71-1bf9a935c8f5>:1) ]] [Op:__inference_train_function_17633]

Function call stack:
train_function


In [ ]:
y_hat = model.predict(X_test)


In [ ]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))


In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)
print("NN-F1 score",metrics.f1_score(pred, test,average='weighted'))

In [ ]:
print(classification_report(y_test, pred, target_names=labels))